## 1. Learn Langchain

https://www.youtube.com/watch?v=nAmC7SoVLd8


In [1]:
from secret_key import openai_key
import os
os.environ['OPENAI_API_KEY'] = openai_key


In [2]:
from langchain.llms import OpenAI
llm = OpenAI(temperature=0.9)
llm("What would be a good company name for a company that makes colorful socks?")

/var/folders/zt/63tm27h17v75fy_47rqppsl40000gn/T/ipykernel_53572/2129428552.py:2: LangChainDeprecationWarning: The class `OpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAI``.
  llm = OpenAI(temperature=0.9)
/var/folders/zt/63tm27h17v75fy_47rqppsl40000gn/T/ipykernel_53572/2129428552.py:3: LangChainDeprecationWarning: The method `BaseLLM.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  llm("What would be a good company name for a company that makes colorful socks?")


'\n\nRainbow Threads Sock Co.'

### 2. PromptTemplate

In [3]:
from langchain.prompts import PromptTemplate

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)
p = prompt_template_name.format(cuisine="Chinese")

print(p)


I want to open a restaurant for Chinese food. Suggest a fancy name for this.


### 3. Chains

In [4]:
from langchain.chains import LLMChain

chain = LLMChain(llm=llm, prompt=prompt_template_name)
chain.run("Mexican")

/var/folders/zt/63tm27h17v75fy_47rqppsl40000gn/T/ipykernel_53572/723744833.py:3: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  chain = LLMChain(llm=llm, prompt=prompt_template_name)
/var/folders/zt/63tm27h17v75fy_47rqppsl40000gn/T/ipykernel_53572/723744833.py:4: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain.run("Mexican")


'\n"El Sabroso Cantina" or "La Cocina de Mexico"'

In [5]:
chain = LLMChain(llm=llm, prompt=prompt_template_name, verbose=True)
chain.run("Mexican")



> Entering new LLMChain chain...
Prompt after formatting:
I want to open a restaurant for Mexican food. Suggest a fancy name for this.

> Finished chain.


'\n"La Cantina de Oro" (The Golden Cantina)'

### 4. SimpleSequentialChain

In [6]:
llm = OpenAI(temperature=0.6)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="""Suggest some menu items for {restaurant_name}"""
)

food_items_chain = LLMChain(llm=llm, prompt=prompt_template_items)

In [7]:
from langchain.chains import SimpleSequentialChain
chain = SimpleSequentialChain(chains = [name_chain, food_items_chain])

content = chain.run("Chinese")
print(content)



1. General Tso's Chicken
2. Mongolian Beef
3. Kung Pao Shrimp
4. Vegetable Fried Rice
5. Sweet and Sour Pork
6. Szechuan Tofu
7. Egg Drop Soup
8. Steamed Dumplings
9. Lemon Chicken
10. Beef and Broccoli
11. Shrimp Lo Mein
12. Hot and Sour Soup
13. Sesame Chicken
14. Garlic Green Beans
15. Honey Walnut Shrimp
16. Vegetable Spring Rolls
17. Singapore Noodles
18. Ma Po Tofu
19. Peking Duck
20. Fortune Cookies for dessert.


### 5. SequentialChain

In [8]:
llm = OpenAI(temperature=0.7)

prompt_template_name = PromptTemplate(
    input_variables =['cuisine'],
    template = "I want to open a restaurant for {cuisine} food. Suggest a fancy name for this."
)

name_chain =LLMChain(llm=llm, prompt=prompt_template_name, output_key="restaurant_name")


In [9]:
llm = OpenAI(temperature=0.7)

prompt_template_items = PromptTemplate(
    input_variables = ['restaurant_name'],
    template="Suggest some menu items for {restaurant_name}."
)

food_items_chain =LLMChain(llm=llm, prompt=prompt_template_items, output_key="menu_items")

In [10]:
from langchain.chains import SequentialChain

chain = SequentialChain(
    chains = [name_chain, food_items_chain],
    input_variables = ['cuisine'],
    output_variables = ['restaurant_name', "menu_items"]
)

In [11]:
chain({"cuisine": "Chinese"})

/var/folders/zt/63tm27h17v75fy_47rqppsl40000gn/T/ipykernel_53572/3925546723.py:1: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  chain({"cuisine": "Chinese"})


{'cuisine': 'Chinese',
 'restaurant_name': '\n\n"Imperial Wok"',
 'menu_items': " \n\n1. General Tso's Chicken \n2. Orange Beef \n3. Vegetable Lo Mein \n4. Shrimp Fried Rice \n5. Kung Pao Tofu \n6. Sweet and Sour Pork \n7. Beef and Broccoli \n8. Sesame Chicken \n9. Szechuan Shrimp \n10. Vegetable Egg Rolls \n11. Crab Rangoon \n12. Mongolian Beef \n13. Honey Walnut Shrimp \n14. Hot and Sour Soup \n15. Garlic Green Beans \n16. Chicken Chow Mein \n17. Spicy Mapo Tofu \n18. Moo Shu Pork \n19. Steamed Dumplings \n20. Pineapple Chicken."}

### 6. Agent

In [12]:
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain.llms import OpenAI
tools =load_tools(["wikipedia","llm-math"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("when was elon musk born? what is his current age in 2025?")
                         

/var/folders/zt/63tm27h17v75fy_47rqppsl40000gn/T/ipykernel_53572/2039906735.py:4: LangChainDeprecationWarning: LangChain agents will continue to be supported, but it is recommended for new use cases to be built with LangGraph. LangGraph offers a more flexible and full-featured framework for building agents, including support for tool-calling, persistence of state, and human-in-the-loop workflows. For details, refer to the `LangGraph documentation <https://langchain-ai.github.io/langgraph/>`_ as well as guides for `Migrating from AgentExecutor <https://python.langchain.com/docs/how_to/migrate_agent/>`_ and LangGraph's `Pre-built ReAct agent <https://langchain-ai.github.io/langgraph/how-tos/create-react-agent/>`_.
  agent = initialize_agent(




> Entering new AgentExecutor chain...
 First, we need to find out when Elon Musk was born.
Action: wikipedia
Action Input: elon musk
Observation: Page: Elon Musk
Summary: Elon Reeve Musk ( EE-lon; born June 28, 1971) is a businessman and entrepreneur known for his leadership of Tesla, SpaceX, X (formerly Twitter), and xAI. Musk has been the wealthiest person in the world since 2021; as of October 2025, Forbes estimates his net worth to be US$500 billion.
Born to a wealthy family in Pretoria, South Africa, Musk emigrated in 1989 to Canada; he had obtained Canadian citizenship at birth through his Canadian-born mother. He received bachelor's degrees in 1997 from the University of Pennsylvania in Philadelphia, United States, before moving to California to pursue business ventures. In 1995, Musk co-founded the software company Zip2. Following its sale in 1999, he co-founded X.com, an online payment company that later merged to form PayPal, which was acquired by eBay in 2002. That year, M

"Elon Musk's current age in 2025 will be 54 years old."

### 7. Google Search API

In [ ]:
import os
from secret_key import serpapi_key
from langchain.agents import initialize_agent, load_tools, AgentType
from langchain.llms import OpenAI

os.environ['SERPAPI_API_KEY'] = serpapi_key
llm = OpenAI(temperature=0)
tools = load_tools(["serpapi","llm-math"], llm=llm)
agent = initialize_agent(
    tools,
    llm,
    agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    verbose=True
)

agent.run("what was the US GDP in 2020? what about 2019? what is the difference between them?")





> Entering new AgentExecutor chain...
 I should use the search engine to find the US GDP for 2020 and 2019.
Action: Search
Action Input: "US GDP 2020" and "US GDP 2019"
Observation: ['U.S. GDP for 2021 was 23.681 trillion US dollars, a 10.9% increase from 2020. U.S. GDP for 2020 was 21.354 trillion US dollars, a 0.86% decline from 2019. GDP ...', 'Current-dollar GDP increased 4.1 percent, or $848.8 billion, in 2019 to a level of $21.43 trillion, compared with an increase of 5.4 percent, ...', 'Gross domestic product (GDP), the featured measure of US output, is the market value of the goods and services produced by labor and property located in the ...', 'In the first quarter, real GDP decreased 0.6 percent (revised). The increase in real GDP in the second quarter primarily reflected a decrease in imports, which ...', 'GDP growth (annual %) - United States. Country official statistics, National Statistical Organizations and/or Central Banks; National Accounts data files.', 'GDP Growth

'The difference between the US GDP in 2020 and 2019 is approximately $76 billion.'